## Задание ##

Вы работаете аналитиком в быстрой доставке SkySamokat.

Периодически клиентам рассылаются пуш-уведомления, в которых описывается специальное предложение недели.
Перед вами список клиентов, которые получали эти пуш-уведомления. По результату рассылки они воспользовались или не воспользовались предложением (поле ***flag_conv***).

Ваши коллеги из отдела маркетинга собираются запустить АБ Тест, чтобы проверить, не будет ли смс-сообщение более эффективным чем пуш-уведомление. Вам необходимо протестировать систему сплитования и рассчитать время, необходимое для расчета эксперимента.

1. Исследуйте зависимость пользования предложением от идентификатора клиента. Видите ли вы какие-то закономерности?
Подсказка
Постройте график зависимости таргет-величины от последней цифры идентификатора клиента. Воспользуйтесь следующим кодом:
**df\['right_symbol'\] = df\['id_client'\].astype(str).str\[-1\]**

2. Сделайте разбиение клиентов по группам (контроль и тест) в зависимости от остатка деления идентификатора на 10. Рассчитайте результаты АА-теста. Наблюдается ли разница?

3. Сделайте разбиение клиентов по группам (контроль и тест) в зависимости от четности идентификатора (остатка от деления идентификатора на 2). Рассчитайте результаты АА-теста. Наблюдается ли разница?

4. Сделайте разбиение клиентов по группам (контроль и тест) с помощью функции sample из библиотеки pandas. По аналогии с расчетом из лекции сделайте 1000 разбиений с помощью sample и рассчитайте статистическую значимость разниц. В скольких процентах случаев наблюдается статистически значимая разница между контролем и тестом в АА-тесте?
**Задача со звездочкой**: проверьте, что это доля (случаев, в которых разница между группами есть) равна 5% на 95%-уровне значимости.

5. Рассчитайте, сколько наблюдений потребуется для проведения АБ Теста, при условии что MDE = 2 процентных пункта (предположите, что альфа и бета установлены на дефолтном уровне и что дисперсия выборки не изменится от воздействия В).

In [ ]:
### Импортируем необходимое

import pandas as pd
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import datetime as dt
import random
import math

df = pd.read_csv('Домашнее задание Бизнес-4 (исходник).csv', sep=',')
df.head()

In [ ]:
### Проверяем данные

df.info()

In [ ]:

df.duplicated().sum()

### df.drop_duplicates(inplace = True) (если необходимо удалить дубликаты и перезаписать датафрейм без дубликатов)

In [ ]:

df.isnull().sum()

### df.dropna(inplace = True) (если необходимо удалить нулы и перезаписать датафрейм без дубликатов). По "умолчанию" удалять нулы можно, если удаляется не более 5% данных.

### Решение ###

In [ ]:
### 1. Исследуйте зависимость пользования предложением от идентификатора клиента. Видите ли вы какие-то закономерности?

df['right_symbol'] = df['id_client'].astype(str).str[-1]
df.head()

In [ ]:

df_agg = df.groupby('right_symbol').agg(conv = ('flag_conv', 'mean')).reset_index()
df_agg.head(10)

In [ ]:

plt.rcParams["figure.figsize"] = (13,4)
fig, ax = plt.subplots()
fig.suptitle('Зависимость результата от № ID')

ax.plot(df_agg['right_symbol'], df_agg['conv'], color = 'green', alpha=.7)
ax.set_xlabel('Посл. цифра ID')
ax.set_ylabel('Конверсия в использование предложения')
plt.show()

на вкидку - можем предположить, что распределение выборки было неверным, на что указывает почти в два раза более высокая конверсия по ID, заказчивающимся на 0

In [ ]:
### 2. Сделайте разбиение клиентов по группам (контроль и тест) в зависимости от остатка деления ID на 10. Рассчитайте АА-тест. Наблюдается ли разница?

df['to_10'] = df['id_client'] % 10
unique_values = df['to_10'].unique()

while True:
    try:
        selected_values = tuple(np.random.choice(unique_values, size = 5, replace = False))
        df['group_10'] = np.where(df['to_10'].isin(selected_values), 'test', 'control')
        r1 = df[df['group_10'] == 'control']['flag_conv']
        r2 = df[df['group_10'] == 'test']['flag_conv']
        alpha = 0.05
        s, p = mannwhitneyu(r1, r2)
        if p > alpha:
            break
        else:
            print('Распределения не равны, сетап(разбиение) для эксперимента выбран некорректно')
            print(f'Значения колонки to_10 для тестовой группы = {selected_values}')
            print()
    except:
        print('Нет данных')
print(f'Распределения равны. P_value = {p}')
print(f'Значения колонки to_10 для тестовой группы = {selected_values}')
print()

In [ ]:

df_info = df.groupby('group_10').agg(cnt_client = ('id_client', 'count'), conv = ('flag_conv', 'mean')).reset_index()
df_info.head()

In [ ]:

s1, p1 = ttest_ind(r1, r2)
print(s1)
alpha = 0.05
if p1 >= alpha:
    print(f'p value = {p1}. Принимаем H0 -> нет стат. значимой разницы между выборками.')
else:
    print(f'p value = {p1}. Принимаем H1 -> есть стат. значимая разница между выборками.')
print()

In [ ]:
### 3. Распределите клиентов по группам (контроль и тест) на основе остатка от деления ID на 2. Рассчитайте результаты АА-теста. Наблюдается ли разница?

df['to_2'] = df['id_client'] % 2
df['group_2'] = np.where(df['to_2'] == 0, 'test', 'control')
df.head()

In [ ]:

df_info1 = df.groupby('group_2').agg(cnt_client = ('id_client', 'count'), conv = ('flag_conv', 'mean')).reset_index()
df_info1.head()

In [ ]:

alpha = 0.05
ss, pp = mannwhitneyu(df[df['group_2'] == 'control']['flag_conv'], df[df['group_2'] == 'test']['flag_conv'])
if pp > alpha:
    print('Распределения равны')
    print('P_value = ',pp)
    print()
else:
    print('Распределения не равны')

In [ ]:

ss1, pp1 = ttest_ind(df[df['group_2'] == 'control']['flag_conv'], df[df['group_2'] == 'test']['flag_conv'])
print(ss1)
alpha = 0.05
if pp1 >= alpha:
    print(f'p value = {pp1}. Принимаем H0 -> нет стат. значимой разницы между выборками.')
else:
    print(f'p value = {pp1}. Принимаем H1 -> есть стат. значимая разница между выборками.')
print()

In [ ]:
### 4. Распределите клиентов по группам (контроль и тест) с помощью функции sample (1000 разбиений) из библиотеки pandas и рассчитайте статистическую значимость разниц.
### В скольких процентах случаев наблюдается статистически значимая разница между контролем и тестом в АА-тесте?
### Задача со звездочкой: проверьте, что это доля (случаев, в которых разница между группами есть) равна 5% на 95%-уровне значимости.

results = []

for i in range(1,1000):
    part_50 = df.sample(frac = 0.5)
    rest_part_50 = df.drop(part_50.index)

    s_a, p_a = ttest_ind(part_50['flag_conv'], rest_part_50['flag_conv'])

    results.append({'iter':i,'stat_crit':s_a,'p_value':p_a})
    df_aa = pd.DataFrame(results)

df_aa.head()

In [ ]:

df_aa[df_aa['p_value']<=0.05].count() / df_aa.count()

In [ ]:

m = df_aa['p_value'].mean()
sig = df_aa['p_value'].std()

print('Среднее = ', m)
print('Сигма = ', sig)

In [ ]:

df_two_sigma = df_aa[(df_aa['p_value'] >= m - 2*sig) & (df_aa['p_value'] <= m + 2*sig)]
print('Доля случаев, в которых разница между группами есть - ', 1 - (df_two_sigma.count()['iter'] / df_aa.count()['iter']))

In [ ]:
### 5. Cколько наблюдений потребуется для АБ Теста (MDE = 2 проц.п, альфа и бета на дефолтном уровне и дисперсия выборки не изменится от воздействия В).

m = df['flag_conv'].mean()
sigma = m*(1-m)
n = 16 * sigma / (0.02**2)

print(f'Требуемое кол-во наблюдений в самой маленькой группе = {n}')
print(f'Общее требуемое кол-во наблюдений в двух группах = {n * 2}')